<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0502Minnichi_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# みんなの日本語関係の処理 データ作成
- date: 2021_0502
- author: 浅川伸一



In [ ]:
%load_ext autoreload
%autoreload 2
#!pwd

import os
import sys
import unicodedata
import glob

import numpy as np
np.set_printoptions(precision=2)  # numpy の表示桁数設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#colab 上で MeCab をインストール際に，次行をコメントアウト
!pip install mecab-python3==0.996.3
import MeCab

wakati = MeCab.Tagger('-Owakati')
yomi = MeCab.Tagger('-Oyomi')

!pip install jaconv
import jaconv

!pip install japanize_matplotlib
import japanize_matplotlib  # matplotlib の日本語表示

In [ ]:
# word2vec データ処理のため gensim を使う
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

#from scipy import stats
#import termcolor

In [ ]:
#GitHub からライブラリをダウンロード
!wget -O ja_util.py https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/2020ccap/ja_util.py
#カレントディレクトリからライブラリを読み込むために path を追加
import sys
sys.path.append(".")
# ja_util は 2021 Apr に作った日本語処理関係
import ja_util   #自作の ja_util を読み込み

In [ ]:
# word2vec データの読み込み
# ローカルディスクから読み込むようになっています。colab でお使いの場合には適宜変更してください
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz

# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります
w2v_base = './'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 

w2v = asakawa_w2v

以下，コードではなく，テキストにしています。データに著作権があるでしょから，保護の意味合いです。

```pytthon
# 岩下先生から頂いた「みんなの日本語」データの読み込み
jlpt_base = '/Users/asakawa/study/2021jlpt'
minnichi_files = sorted(glob.glob(os.path.join(jlpt_base, 'MINNICHI_*.txt')))
print(minnichi_files)

# みんなの日本語テキストを読み込み
minnichi_text = {}
text_data = []
for file in minnichi_files:
    fname = os.path.split(file)[-1].split('.')[0]

    if not fname in minnichi_text:
        minnichi_text[fname] = []
    #txt = []
    with open(file,'r') as f:
        texts = f.readlines()
        
        for txt in texts:
            txt = unicodedata.normalize("NFC", txt).strip()  # 全角記号を半角へ置換
            #txt = ja_util.normalize_neologd(txt)
            if len(txt) > 1:
                minnichi_text[fname].append(txt)
                text_data.append(wakati.parse(txt).strip().split())

#「いっしょにごはん」を 「いっしょ」「に」「ごはん」に修正
text_data[120] = ['今度', 'は', 'わたし', 'の', 'うち', 'へ', '来', 'ませ', 'ん', 'か', '。', 'ここ', 'の', '野菜', 'は', 'おいしい', 'です', 'よ', '。', 'いっしょ', 'に', 'ごはん', 'を', '食べ', 'ましょ', 'う', '。']

# 「こいぬ」を [子犬] に修正
#text_data[207] = ['D', ':', 'げんき', 'な', '子犬', 'が', '5', 'ひきい', 'ます', '。', 'しろい', 'の', 'が', '3', 'びき', 'と', 'くろい', 'の', 'が', '2', 'ひき', 'です', '。', 'もらっ', 'て', 'ください', '。']


# 229 「大人しか」 を 「大人」 「しか」に修正
text_data[229] = ['それで', '、', 'お酒', 'の', '販売', '機', 'は', '夜', '11時', 'から', '朝', '5時', 'まで', '止め', 'て', 'ある', '。', 'たばこ', 'は', '「', 'タスポ', '」', 'という', 'カード', 'を', '機械', 'に', 'タッチ', 'し', 'なけれ', 'ば', '、', '買え', 'ない', '。', 'この', 'カード', 'は', '大人', 'しか', '持て', 'ない', '。', 'また', '、', '最近', 'で', 'は', '、', '販売', '機', 'の', 'ほとんど', 'が', '節電', 'タイプ', 'に', 'なっ', 'て', 'いる', '。', '1991年', 'から', '2013年', 'まで', 'に', '、', '飲み物', 'の', '販売', '機', 'が', '使う', '電力', 'は', '75', '％', '減っ', 'た', '。']

# 338 「わたしたち」 を「わたし」「たち」に修正
text_data[338] = ['次に', 'コーヒー', 'を', '飲む', 'と', '、', 'リラックス', 'する', 'こと', 'が', 'でき', 'ます', '。', 'です', 'から', 'わたし', 'たち', 'は', '喫茶店', 'で', '友達', 'と', '話す', 'とき', '、', '仕事', 'が', '終わっ', 'て', '少し', '休む', 'とき', '、', 'よく', 'コーヒー', 'を', '飲み', 'ます', '。']

# 「かた」「かな」 を修正
text_data[-14] = ['かたかな', 'の', 'ことば', 'は', '英語', 'から', '来', 'た', 'ことば', 'が', '多い', 'です', '。', '日本', 'で', '作っ', 'た', 'かたかな', 'の', 'ことば', 'も', 'たくさん', 'あり', 'ます', 'が', '、', '英語', 'で', 'は', 'あり', 'ませ', 'ん', '。', 'です', 'から', '、', '英語', 'が', 'わかる', '外国人', 'も', '意味', 'が', 'わかり', 'ませ', 'ん', '。']    

# [.日本] を [.] [日本] に修正
text_data[-11] = ['2', '.', '日本', 'の', '車', 'は', 'ハンドル', 'が', '右', 'に', 'あり', 'ます', '。']

# 以上の修正で，word2vec にエントリのない語彙は，以下のとおり `!!??`, `かたかな`, `こいぬ`, `ふろ場`, `弁当屋`, `赤繁`

with open('minnichi.txt', 'w') as f:
    for line in text_data:
        for word in line:
            f.write(f'{word} ')
        f.write('\n')

```

In [ ]:
#上で作成した minnchi.txt を upload してださい。
from google.colab import files
files.upload()

{}

In [ ]:
with open('minnichi.txt', 'r') as f:
    X = f.readlines()

text_data = [line.strip().split() for line in X]

minnichi_freq = {}
for line in text_data:
    for word in line:
        if not word in minnichi_freq:
            minnichi_freq[word] = 1
        else:
            minnichi_freq[word] += 1

minnichi_words = list(sorted(minnichi_freq))

minnichi_words = sorted(minnichi_freq)
XXX = np.array([minnichi_freq[w] for w in minnichi_freq])
plt.plot(np.sort(XXX)[::-1])

minnichi_freq = {}
for line in text_data:
    for word in line:
        if not word in minnichi_freq:
            minnichi_freq[word] = 1
        else:
            minnichi_freq[word] += 1

minnichi_words = list(sorted(minnichi_freq))

minnichi_words = sorted(minnichi_freq)
XXX = np.array([minnichi_freq[w] for w in minnichi_freq])
plt.plot(np.sort(XXX)[::-1])
#XX = [line.strip().split() for line in X]
#print(XX[-14])

In [ ]:
for word in minnichi_words:
    if word not in w2v:
        print(word)

In [ ]:
# みんなの日本語全テキストをモーラへ変換
min_mora = {}
for word in minnichi_words:
    yomi = ja_util.mora_wakati().getPronunciation(word)
    min_mora[word] = {'yomi':yomi,
                      'id': minnichi_words.index(word),
                      'freq': minnichi_freq[word]
                     }


In [ ]:
print(min_mora)